In [1]:
import numpy as np

In [86]:
class Expression:
    def __init__(self, ele_func, sub_expr1, sub_expr2=None):
        self._ele_func  = ele_func
        self._sub_expr1 = sub_expr1
        self._sub_expr2 = sub_expr2
    
    def evaluation_at(self, val_dict):
        
        # self._sub_expr2 is None implies that self._ele_func is an unary operator
        if self._sub_expr2 is None: 
            return self._ele_func.evaluation_at(
                self._sub_expr1, val_dict)
        
        # self._sub_expr2 not None implies that self._ele_func is a binary operator
        else:
            return self._ele_func.evaluation_at(
                self._sub_expr1, self._sub_expr2, val_dict)
    
    def derivative_at(self, var, val_dict):
        
        if var is self: return 1.0
        
        # sub_expr2 is None implies that _ele_func is an unary operator
        if self._sub_expr2 is None:
            return self._ele_func.derivative_at(
                self._sub_expr1, var, val_dict)
        
        # sub_expr2 not None implies that _ele_func is a binary operator
        else:
            return self._ele_func.derivative_at(
                self._sub_expr1, self._sub_expr2, var, val_dict)
    
    def __add__(self, another):
        if isinstance(another, Expression):
            return Expression(Add, self, another)
        # if the other operand is not an Expression, then it must be a number
        # the number then should be converted to a Constant
        else:
            return Expression(Add, self, Constant(another))
    
    def __radd__(self, another):
        if isinstance(another, Expression):
            return Expression(Add, another, self)
        else:
            return Expression(Add, Constant(another), self)
    
    def __sub__(self, another):
        if isinstance(another, Expression):
            return Expression(Sub, self, another)
        else:
            return Expression(Sub, self, Constant(another))
    
    def __rsub__(self, another):
        if isinstance(another, Expression):
            return Expression(Sub, another, self)
        else:
            return Expression(Sub, Constant(another), self)
        

    def __mul__(self, another):
        if isinstance(another, Expression):
            return Expression(Mul,self,another)
        else:
            return Expression(Mul, self, Constant(another))

    def __rmul__(self, another):
        if isinstance(another, Expression):
            return Expression(Mul,another,self)
        else:
            return Expression(Mul, Constant(another),self)
    
    def __truediv__(self, another):
        if isinstance(another, Expression):
            return Expression(Div,self,another)
        else:
            return Expression(Div, self, Constant(another))

    def __rtruediv__(self, another):
        if isinstance(another, Expression):
            return Expression(Div,another,self)
        else:
            return Expression(Div, Constant(another),self)
    
    def __pow__(self,another):
        if isinstance(another, Expression):
            return Expression(Pow,self,another)
        else:
            return Expression(Pow, self, Constant(another))
    
    def __rpow__(self,another):
        if isinstance(another, Expression):
            return Expression(Pow,another,self)
        else:
            return Expression(Pow, Constant(another),self)

In [87]:
class Variable(Expression):
    def __init__(self):
        return
    
    def evaluation_at(self, val_dict):
        return val_dict[self]
    
    def derivative_at(self, var, val_dict):
        return 1.0 if var is self else 0.0

In [88]:
class Constant(Expression):
    def __init__(self, val):
        self.val = val
        
    def evaluation_at(self, val_dict):
        return self.val
    
    def derivative_at(self, var, val_dict):
        return 0.0

In [89]:
class Add:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) + \
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) + \
               sub_expr2.derivative_at(var, val_dict)

In [90]:
class Pow:
    
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) **\
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    #f(x)^g(x) * g‘(x)  * ln( f(x) )+ f(x)^( g(x)-1 ) * g(x) * f’(x) 
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return  sub_expr1.evaluation_at(val_dict)** \
                sub_expr2.evaluation_at(val_dict)* \
                sub_expr2.derivative_at(var, val_dict)*\
                np.log(sub_expr1.evaluation_at(val_dict))+ \
                sub_expr1.evaluation_at(val_dict) **\
                (sub_expr2.evaluation_at(val_dict)-1)*\
                sub_expr2.evaluation_at(val_dict)*\
                sub_expr1.derivative_at(var, val_dict)

In [91]:
class Sub:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) - \
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) - \
               sub_expr2.derivative_at(var, val_dict)

In [92]:
class Mul:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) *\
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) * \
               sub_expr2.evaluation_at(val_dict)+ \
               sub_expr1.evaluation_at(val_dict) *\
               sub_expr2.derivative_at(var, val_dict)

In [93]:
class Div:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) /\
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return  sub_expr1.derivative_at(var, val_dict) / \
                sub_expr2.evaluation_at(val_dict)+ \
                sub_expr1.evaluation_at(val_dict) *\
                sub_expr2.derivative_at(var, val_dict)/\
                sub_expr2.evaluation_at(val_dict)/\
                sub_expr2.evaluation_at(val_dict)

In [94]:
class Exp:
    @staticmethod
    def evaluation_at(sub_expr1, val_dict):
        return np.exp(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) * np.exp(sub_expr1.evaluation_at(val_dict))

In [95]:
def exp(expr):
    return Expression(Exp, expr)

In [96]:
class Sin:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return np.sin(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        return sub_expr1.derivative_at(var, val_dict)*np.cos(sub_expr1.evaluation_at(val_dict)) 

In [97]:
def sin(expr):
    return Expression(Sin,expr)

In [98]:
a = Variable()
b = Variable()
c = a+b
f1 = sin(a+c)

In [99]:
f2 = sin(a*b)

In [100]:
f2.evaluation_at({a:1,b:2})

0.90929742682568171

In [101]:
f2.derivative_at(a,{a:2,b:2})

-1.3072872417272239

In [17]:
#expected derivative
np.cos(4)*2

-1.3072872417272239

In [18]:
# function output
## may need to improve the interface.
f1.evaluation_at({a:1, b: 2.0})

-0.7568024953079282

In [19]:
# expected value
np.sin(1+2+1)

-0.7568024953079282

In [20]:
f1.derivative_at(c,{a: 1.0, b: 1.0})

-0.98999249660044542

$$\frac{df}{dc} = cos(a+c) = cos(3)$$

In [21]:
np.cos(3)

-0.98999249660044542

Let $f(a, b, c) = e^{a-b+c}$, at $(a, b, c) = (1.0, 2.0, 3.0)$, that should be $e^{2} \approx 7.389$.

In [22]:
a = Variable()
b = Variable()
c = Variable()
f = exp(a-b+c)

In [23]:
f.evaluation_at({a: 1.0, b: 2.0, c: 3.0})

7.3890560989306504

In [24]:
class Cos:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return np.cos(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        return -sub_expr1.derivative_at(var, val_dict)*np.sin(sub_expr1.evaluation_at(val_dict)) 

In [25]:
def cos(expr):
    return Expression(Cos,expr)

In [26]:
a = Variable()
b = Variable()
c = a+b
f1 = cos(a+c)
f2 = cos(a*b)

In [27]:
print(f1.evaluation_at({a:1.0, b: 2.0}))
print(f2.evaluation_at({a:1.0,b:2}))

-0.653643620864
-0.416146836547


In [28]:
# expected
print(np.cos(4))
print(np.cos(2))

-0.653643620864
-0.416146836547


In [29]:
f1.derivative_at(a,{a:1.0, b: 2.0})

1.5136049906158564

In [30]:
# expected
-np.sin(1+3)*2

1.5136049906158564

In [31]:
f2.derivative_at(a,{a:2,b:2})

1.5136049906158564

In [32]:
#expected
print(-np.sin(2*2)*2)

1.51360499062


In [33]:
class Tan:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return np.tan(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        return sub_expr1.derivative_at(var, val_dict)*(1/np.cos(2*sub_expr1.evaluation_at(val_dict)))
def tan(expr):
    return Expression(Tan,expr)

In [34]:
a = Variable()
b = Variable()
c = a*b
f = tan(c*b)

In [35]:
f.evaluation_at({a:1,b:2})

1.1578212823495775

In [36]:
#expected
np.tan(4)

1.1578212823495775

In [37]:
f.derivative_at(c,{a:1,b:2})

-13.745701273380741

In [38]:
# expected derivative
2*(1/np.cos(4*2))

-13.745701273380741

In [39]:
class Cotan:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return 1/np.tan(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
    
        return -sub_expr1.derivative_at(var, val_dict)*(1/np.sin(sub_expr1.evaluation_at(val_dict))**2)

def cotan(expr):
    return Expression(Cotan,expr)

In [40]:
a = Variable()
b = Variable()
c = a*b
f = cotan(c*b)

In [41]:
f.evaluation_at({a:1,b:2})

0.86369115445061673

In [42]:
1/np.tan(4)

0.86369115445061673

In [43]:
f.derivative_at(c,{a:1,b:2})

-3.491924820552478

In [44]:
-(1/(np.sin(4)**2))*2

-3.491924820552478

In [45]:
class Sec:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return 1/np.cos(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x=sub_expr1.evaluation_at(val_dict)
        return sub_expr1.derivative_at(var, val_dict)*np.tan(x)*(1/np.cos(x))
def sec(expr):
    return Expression(Sec,expr) 

In [46]:
a = Variable()
b = Variable()
c = a*b
f = sec(c*b)

In [47]:
print(f.evaluation_at({a:1,b:2}))
print(1/np.cos(4))

-1.52988565647
-1.52988565647


In [48]:
print(f.derivative_at(c,{a:1,b:2}))
print(np.tan(4)*(1/np.cos(4))*2)

-3.54266834524
-3.54266834524


In [49]:
class Csc:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return 1/np.sin(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x=sub_expr1.evaluation_at(val_dict)
        return sub_expr1.derivative_at(var, val_dict)*(1/np.tan(x))*(1/np.sin(x))
def csc(expr):
    return Expression(Csc,expr) 

In [50]:
a = Variable()
b = Variable()
c = a*b
f = csc(c*b)

In [51]:
print(f.evaluation_at({a:1,b:2}))
print(1/np.sin(4))

-1.32134870881
-1.32134870881


In [52]:
print(f.derivative_at(c,{a:1,b:2}))
print((1/np.tan(4))*(1/np.sin(4))*2)

-2.28247438349
-2.28247438349


In [53]:
class Sinh:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return np.sinh(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x=sub_expr1.evaluation_at(val_dict)
        return sub_expr1.derivative_at(var, val_dict)*np.cosh(x)
def sinh(expr):
    return Expression(Sinh,expr) 

In [54]:
a = Variable()
b = Variable()
c = a*b
f = sinh(c*b)

In [55]:
assert(f.evaluation_at({a:1,b:2})==np.sinh(4))

In [56]:
assert(f.derivative_at(c,{a:1,b:2})==np.cosh(4)*2)

In [57]:
class Cosh:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return np.cosh(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x=sub_expr1.evaluation_at(val_dict)
        return sub_expr1.derivative_at(var, val_dict)*np.sinh(x)
def cosh(expr):
    return Expression(Cosh,expr) 

In [58]:
a = Variable()
b = Variable()
c = a*b
f = cosh(c*b)
assert(f.evaluation_at({a:3,b:2})==np.cosh(12))
assert(f.derivative_at(c,{a:3,b:2})==np.sinh(12)*2)

In [63]:
class Tanh:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return np.sinh(x)/np.cosh(x)
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return sub_expr1.derivative_at(var, val_dict)*(1/np.cosh(x)**2)
    
def tanh(expr):
    return Expression(Tanh,expr) 

In [64]:
a = Variable()
b = Variable()
c = a*b
f = tan(c*b)
assert(f.evaluation_at({a:3,b:2})==np.sin(12)/np.cos(12))
assert(f.derivative_at(c,{a:3,b:2})==1/np.cos(24)*2)

In [65]:
class Csch:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return 1/np.sinh(x)
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        # -csch(x)*cot(x)
        d=-(1/np.sinh(x))*(np.cosh(x)/np.sinh(x))
        return sub_expr1.derivative_at(var, val_dict)*d
def csch(expr):
    return Expression(Csch,expr) 

In [66]:
a = Variable()
b = Variable()
c = a*b
f = csch(c*b)
print(f.evaluation_at({a:3,b:2}))
print(1/np.sinh(12))
print(f.derivative_at(c,{a:3,b:2}))
print(-(np.cosh(12)/np.sinh(12))*(1/np.sinh(12))*2)

1.22884247071e-05
1.22884247071e-05
-2.45768494161e-05
-2.45768494161e-05


In [67]:
class Sech:
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return 1/np.cosh(x)
    
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        # -sech(x)tanh(x)
        d=-(1/np.cosh(x))*(np.sinh(x)/np.cosh(x))
        return sub_expr1.derivative_at(var, val_dict)*d
def sech(expr):
    return Expression(Sech,expr)     

In [68]:
a = Variable()
b = Variable()
c = a*b
f = sech(c*b)
print(f.evaluation_at({a:2,b:1}))
print(1/np.cosh(2))
print(f.derivative_at(c,{a:2,b:1}))
print(-(np.sinh(2)/np.cosh(2))*(1/np.cosh(2))*1)

0.265802228834
0.265802228834
-0.256240679442
-0.256240679442


In [69]:
class Coth:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return np.cosh(x)/np.sinh(x)
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        #-csch^2(x)
        return -sub_expr1.derivative_at(var, val_dict)*(1/np.sinh(x)**2)

def coth(expr):
    return Expression(Coth,expr)    

In [70]:
a = Variable()
b = Variable()
c = a*b
f = coth(c*b)
print(f.evaluation_at({a:3,b:2}))
print(np.cosh(12)/np.sinh(12))
print(f.derivative_at(c,{a:3,b:2}))
print(-(1/np.sinh(12))**2*2)

1.00000000008
1.00000000008
-3.02010763565e-10
-3.02010763565e-10


In [71]:
class Arcsin:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return np.arcsin(x)
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        d = 1/np.sqrt(1-x**2)
        #1/sqrt(1-x^2)
        return sub_expr1.derivative_at(var, val_dict)*d

def arcsin(expr):
    return Expression(Arcsin,expr)

In [72]:
a = Variable()
b = Variable()
c = a*b
f = arcsin(c*b)
print(f.evaluation_at({a:0.2,b:0.5}))
print(np.arcsin(0.05))
print(f.derivative_at(c,{a:0.2,b:0.5}))
print((1/np.sqrt(1-(0.2*0.5*0.5)**2))*0.5)

0.0500208568058
0.0500208568058
0.500626174322
0.500626174322


In [73]:
class Arccos:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return np.arccos(x)
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        d = 1/np.sqrt(1-x**2)
        #-1/sqrt(1-x^2)
        return -sub_expr1.derivative_at(var, val_dict)*d

def arccos(expr):
    return Expression(Arccos,expr)

In [74]:
a = Variable()
b = Variable()
c = a*b
f = arccos(c*b)
print(f.evaluation_at({a:0.2,b:0.5}))
print(np.arccos(0.05))
print(f.derivative_at(c,{a:0.2,b:0.5}))
print((-1/np.sqrt(1-(0.2*0.5*0.5)**2))*0.5)

1.52077546999
1.52077546999
-0.500626174322
-0.500626174322


In [75]:
class Arctan:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return np.arctan(x)
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        d = 1/(1+x**2)
        #1/1-x^2
        return sub_expr1.derivative_at(var, val_dict)*d

def arctan(expr):
    return Expression(Arctan,expr)

In [76]:
a = Variable()
b = Variable()
c = a*b
f = arctan(c*b)
print(f.evaluation_at({a:2,b:3}))
print(np.arctan(18))
print(f.derivative_at(c,{a:2,b:3}))
print((1/(18**2+1))*3)

1.51529782155
1.51529782155
0.00923076923076923
0.00923076923076923


$\dfrac{\partial f}{\partial b} = -e^{a-b+c}$, at $(a, b, c) = (1.0, 2.0, 3.0)$, that should be $-e^{2} \approx -7.389$.

In [77]:
f.derivative_at(b, {a: 1.0, b: 2.0, c: 3.0})

0.23529411764705882

$\dfrac{\partial f}{\partial a} = e^{a-b+c}$, at $(a, b) = (1.0, 2.0, 3.0)$, that should be $e^{2} \approx 7.389$.

In [74]:
f.derivative_at(a, {a: 1.0, b: 2.0, c: 3.0})

-4.0
-1.1455000338086134


13.893553038339718

Let $g(x, y) = x + e^{y-1}$, at $(x, y) = (1.0, 2.0)$, that should be $1+e \approx 3.718$.

In [75]:
x = Variable()
y = Variable()
g = x + exp(y-1)

In [76]:
g.evaluation_at({x: 1.0, y: 2.0})

3.718281828459045

$\dfrac{\partial g}{\partial x} = 1$

In [77]:
g.derivative_at(x, {x: 1.0, y: 2.0})

1.0

$\dfrac{\partial g}{\partial y} = e^{y-1}$, at $(x, y) = (1.0, 2.0)$, that should be $e^{1} \approx 2.718$.

In [78]:
g.derivative_at(y, {x: 1.0, y: 2.0})

2.718281828459045

#### Test for Multiple

In [79]:
x = Variable()
y = Variable()
g = x*y
f = x/y

In [80]:
g.evaluation_at({x: 2.0, y: 2.0})

4.0

In [81]:
g.derivative_at(x, {x: 1.0, y: 2.0})

2.0

In [82]:
f.evaluation_at({x: 3.0, y: 2.0})

1.5

In [53]:
f.derivative_at(y, {x: 4.0, y: 2.0})

1.0

In [54]:
a = 3.4

In [24]:
2/3

0.6666666666666666

In [107]:
x=Variable()
f=exp(sin(x)) - cos(x**0.5) * sin((cos(x)**2.0 + x**2.0)**0.5)

### Prepare for nonlinear solver

tol = 1.0e-06 # Solver tolerance
nmax = 25 # Maximum number of nonlinear iterations

nli = 0 # Nonlinear iteration counter

print("nli        x           dx    ") # Print-out information
while True:
    evalf = exp(sin(x)) - cos(x**0.5) * sin((cos(x)**2.0 + x**2.0)**0.5) # Get function value
    dx = -evalf.evaluation_at({x:2}) / evalf.derivative_at(x,{x:2}) # Update step
    x = x + dx # Update solution
    #print("{0}    {1:8.6f}     {2:8.6e}".format(nli+1, x, dx))

    # Check for convergence
    if abs(dx) <= tol:
        print("Found solution after {} iterations.".format(nli+1))
        print("There is a root at x = {0:6.4f}.".format(x))
        break

    # Check iteration count
    if nli > nmax:
        print("Exceeded allowable max iterations without finding a root.")
        break

    nli += 1 # Increment nonlinear iteration count

nli        x           dx    


/Users/ting/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in log


KeyError: <__main__.Variable object at 0x10f585908>